In [ ]:
!git clone https://github.com/ethanyanjiali/minChatGPT.git

Cloning into 'minChatGPT'...
remote: Enumerating objects: 315, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 315 (delta 19), reused 4 (delta 0), pack-reused 276
Receiving objects: 100% (315/315), 8.91 MiB | 12.85 MiB/s, done.
Resolving deltas: 100% (195/195), done.


In [4]:
cd /data/abhishek/minChatGPT/src

/data/abhishek/minChatGPT/src


In [ ]:
!python train_sft.py -n exp_name -b 3

Loading EYLSFTStaticDataset train split
Loaded 19934053 tokens from 84576 examples.
Loading EYLSFTStaticDataset test split
Loaded 844060 tokens from 3451 examples.
/home/abhishek/.conda/envs/mingpt2/lib/python3.8/site-packages/torchinfo/torchinfo.py:477: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  action_fn=lambda data: sys.getsizeof(data.storage()),
Layer (type:depth-idx)                                  Output Shape              Param #
GPT                                                     [1, 1024, 50257]          --
├─TransformerDecoder: 1-1                               [1, 1024, 1024]           --
│    └─Embedding: 2-1                                   [1, 1024, 1024]           51,463,168
│    └─Embedding: 2-2                              

In [ ]:
!python train_rm.py -b 2 -n exp_name -p "/data/abhishek/minChatGPT/src/runs/sft_exp_name_202406262347/sft_exp_name_202406262347_final.pt"

/data/abhishek/minChatGPT/src/gpt.py:244: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location="cpu")
Found cached dataset parquet (

In ppo training, I stopped the training after 30K steps.

In [ ]:
!python train_ppo.py -b 1 -n exp_name -a "runs/sft_exp_name_202406262347/sft_exp_name_202406262347_final.pt" -c "runs/rm_exp_name_202406271127/rm_exp_name_202406271127_step30000.pt" -s naive

/data/abhishek/minChatGPT/src/gpt.py:484: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location="cpu")
/data/abhishek/minChatGPT/src/

# 1. Connect to GPU for faster inference
# 2. Set your own question in the form to compare the answer from PPO model and Vanilla GPT-2 medium model
# 3. Loading model too many times may crash and OOM

In [1]:
%%capture
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

#@title Download minChatGPT and dependencies
!git clone https://github.com/ethanyanjiali/minChatGPT.git
!pip install accelerate datasets loralib tokenizers tiktoken torchinfo transformers sentencepiece

In [8]:
#@title Initialize Models
import sys
sys.path.append("./minChatGPT/src")

import torch
import tiktoken
from transformers import GPT2LMHeadModel, AutoTokenizer
from huggingface_hub import hf_hub_download
from gpt import GPT
from configs import get_configs
from transformers import GPT2LMHeadModel, AutoTokenizer
from tempfile import NamedTemporaryFile
import torch

def prepare_gpt2_input(prompt, device):
    enc = tiktoken.get_encoding("gpt2")
    encode = lambda s: enc.encode(s, allowed_special={"<|endoftext|>"})
    decode = lambda l: enc.decode(l)
    indices = encode(prompt)
    x = (torch.tensor(indices, dtype=torch.long, device=device)[None, ...])
    return x, decode


def generate_gpt2(model, prompt, device, samples=2):
    model.eval()
    model.to(device)
    max_new_tokens = 100
    temperature = 0.9
    top_k = 200
    x, decode = prepare_gpt2_input(prompt, device)

    generations = []
    for k in range(samples):
        y = model.generate(x,
                           max_new_tokens,
                           temperature=temperature,
                           top_k=top_k)
        generations.append(decode(y[0].tolist()))
    return generations

filepath = hf_hub_download(repo_id="abhi1304/GPT2-RLHF", filename="ppo_exp_gpt2medium.pt")
# filepath="/data/abhishek/minChatGPT/src/runs/ppo_exp_name_202406290127/ppo_exp_name_202406290127_critic_step21000.pt"
with NamedTemporaryFile("w") as fp:
  state = {"model_state_dict": torch.load(filepath)}
  torch.save(state, fp.name)
  cfg = get_configs("gpt2-medium")
  ppo_model = GPT.from_checkpoint(cfg, fp.name)

tokenizer = AutoTokenizer.from_pretrained("gpt2")
# vanilla_model = GPT2LMHeadModel.from_pretrained("gpt2-medium")

from transformers import pipeline
generator = pipeline('text-generation', model='gpt2-medium')

/tmp/ipykernel_569529/887463640.py:44: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = {"model_state_dict": torch.load(filepath)}


In [14]:
#@title Generate Response
def trim_output(text, prompt):
    text = text[len(prompt):]
    return text.split("\n\n")[0]

device = "cuda" if torch.cuda.is_available() else "cpu"
# Question = "What are the good movies in the history?" #@param {type:"string"}
Question="What are the best colleges in india for engineering?"
prompt = f"""Human: {Question}

Assistant:
"""
with torch.inference_mode():
  ppo_generation = generate_gpt2(ppo_model, prompt, device, samples=1)
  # input_ids = tokenizer.encode(prompt, return_tensors='pt')
  # attention_mask = torch.ones_like(input_ids)
  # Generate text with the model
  # output = vanilla_model.generate(input_ids=input_ids,
  #                         attention_mask=attention_mask,
  #                         max_length=len(prompt)+100,
  #                         pad_token_id=tokenizer.eos_token_id,
  #                         eos_token_id=tokenizer.eos_token_id,
  #                         do_sample=True,
  #                         top_k=50)
  # Decode the output tokens back to text
  # output_text = tokenizer.decode(output[0], skip_special_tokens=True)[0]

output_text = generator(
    prompt,
    max_length=len(prompt)+100,
    num_return_sequences=1,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)[0]["generated_text"]

print()
print("\033[94mPPO output: \n")
print(ppo_generation[0])
print()
print("\033[1m--------------")
print()
print("\033[91mVanilla output: \n")
print(output_text)



PPO output: 

Human: What are the best colleges in india for engineering?

Assistant:

There are a lot of good colleges in India for engineering, if you are interested in engineering you should definitely check out the following colleges:

M.M. Khan University (M.M. Khan is known for producing some of the earliest computer scientists and engineers)

Calcutta University (also known for producing some of the earliest mechanical engineers)

RUGrad University (also known for producing some of the earliest computer scientists and engineers)

The University of Hyderabad

--------------

Vanilla output: 

Human: What are the best colleges in india for engineering?

Assistant:

It depends on the institution and the quality of the faculty and staff.

In Chennai it's most attractive since the student population is more educated, and the college population is the same.

University: The students are too smart for Tamil Nadu State University (NTSU) and Hyderabad University.

Assistant: A lot of In